In [19]:
from neo4j import GraphDatabase
import os 
from langchain.graphs import Neo4jGraph

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"

uri = os.getenv('NEO4J_URI')
user_name = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph = Neo4jGraph(
    url="neo4j+s://81acc431.databases.neo4j.io", 
    username="neo4j", 
    password="bnY_Fbfk3k7MsBPgBxgtUPsRnPPJnUYi7zk1c8azGyE"
)

In [22]:
import requests

url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/da8882249af3e819a80debf3160ebbb3513ee962/microservices.json"
import_query = requests.get(url).json()['query']
graph.query(import_query)


[]

In [23]:
graph.query("RETURN 1 AS test")


[{'test': 1}]

In [ ]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings import HuggingFaceEmbeddings


vector_index = Neo4jVector.from_existing_graph(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)